# TP5 : Visualisation et recommandation

## Exercice 1 : Réduction de dimension

**Jeux de données :**
- Swiss Roll http://dac.lip6.fr/master/wp-content/uploads/2017/09/swiss_roll.csv
- Decathlon http://dac.lip6.fr/master/wp-content/uploads/2017/09/decathlon.csv (supprimer les
variables qualitatives, et renverser les temps de façon à avoir une valeur élevée pour les meilleurs
temps)
- MNIST

### Question 1
**Implémentez le modèle t-SNE.**

Implémentation inspirée de [Visualizing Data using t-SNE](http://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf)

Dans mon implémentation, je calcule d'abord les distances entre les vecteurs $x_i$, puis toutes les probabilités conditionnelles et jointes pour les vecteurs $x_i$. Cela prend de la mémoire, mais accélère les calculs.

In [ ]:
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse, optimize
import itertools
from sklearn import datasets
from multiprocessing import Pool
%matplotlib inline


class TSNE():
    """ Implementation t-SNE to reduce dimension """   
    def __init__(self, n_components=2, perplexity=30.0, n_iter=1000, learning_rate=200.0, n_jobs=4):
        """ Constructor
        :param n_components: Number of dimensions in the output space
        param perplexity: Related to the number of nearest neighbours used
        param n_iter: Number of iterations
        param learning_rate: Coefficient that multiplies the gradient at each iteration
        param n_jobs: Number of processes spawn in the computations"""
        # Parameters:
        self.n_components  = n_components
        self.perplexity    = perplexity
        self.n_iter        = n_iter
        self.learning_rate = learning_rate
        self.n_jobs        = n_jobs
        
        # Memoization:
        self.X         = None # Used to have a simple function find_sigma
        self.distances = None # squared distances, upper triangular matrix
        self.joint_p   = None # self.joint_p[i,j] = p(i, j)
        self.cond_p    = None # self.cond_p[i,j] = p(j|i), upper triang. matrix
        self.sigma     = None # Array of sigmas_i
    
    def get_dist(self, X, i, j):
        """ Return the squared distance between two vectors. """
        if self.distances is None:
            print("Initializing distances matrix")
            self.distances = np.zeros((X.shape[0], X.shape[0]))
            for k,l in itertools.combinations(range(X.shape[0]), 2):
                self.distances[k,l] = sum((X[k] - X[l])**2)
            self.distances = sparse.triu(self.distances, format="lil")
            print("Finished computing distances matrix")
        
        if i>j:
            i,j = j,i
        return self.distances[i,j]
        
    def get_cond_p(self, i, j):
        """ Return p(j|i). Find the sigma that fit the desired perplexity.
        Create the arrays of conditional probas and sigmas if they don't exist"""
        # Create the matrix if it doesn't exist:
        if self.cond_p is None:
            print("Initializing conditional probas matrix")
            N = X.shape[0] # number of vectors
            print("Trigger get_dist")
            self.get_dist(X, 0, 1)
            self.cond_p = np.zeros((N, N))
            pool = Pool(processes=self.n_jobs)
            self.sigma = pool.map(self.find_sigma, range(N))
#             print("Sigmas found: ", self.sigma)
            for k in range(N):
                # Compute all conditional probabilities
                denom = sum([np.exp(-self.get_dist(X, k, m)/(2 * self.sigma[k]**2)) for m in range(N)
                            if m != k])
                for l in range(N):
                    if l == k:
                        self.cond_p[k, l] = 0
                    else:
                        numerator = np.exp(-self.get_dist(X, i, j)/(2 * self.sigma[k]**2))
                        self.cond_p[k,l] = numerator/denom

            print("Finished computing conditional probas matrix.")
        
        return self.cond_p[i, j]
    
    def compute_conditional_p(self, X, i, j, sigma_i):
        """Returns p(j|i), used to find the sigmas that fit the perplexity"""
        if i == j:
            return 0
        numerator = np.exp(-self.get_dist(X, i, j)/(2 * sigma_i**2))
        denom = sum([np.exp(-self.get_dist(X, i, k)/(2 * sigma_i**2)) for k in range(X.shape[0])
                    if k != i])
        return numerator/denom    
    
    def find_sigma(self, i, max_iter=100):
        """Find sigma component that gives the wanted perplexity"""
#         print("Find sigma component #%d to achieve a perplexity of %f" % (i, self.perplexity))
#         print("Find sigmin...")
        sigmin = 1
        n = 0
        while n < max_iter and self.compute_perplexity(X, i, sigmin) > self.perplexity:
            sigmin /= 2
            n += 1
#         print("Found: %f" % sigmin)
            
#         print("Find sigmax...")
        sigmax = 10
        n = 0
        while n < max_iter and self.compute_perplexity(X, i, sigmax) < self.perplexity:
            sigmax *= 2
            n += 1
#         print("Found: %f" % sigmax)
#         print("Calling optimize.bisect")
        f = lambda s: self.compute_perplexity(X, i, s) - self.perplexity
        sigma_i = optimize.bisect(f, sigmin, sigmax, maxiter=max_iter, rtol=1e-5)
#         print("sigma_i = %f" % sigma_i)
        return sigma_i

    def compute_perplexity(self, X, i, sigma_i):
        """ Returns the perplexity"""
#         print("Compute perplexity(i=%d, sigma=%f)" % (i, sigma_i))
        N = X.shape[0]
        h = 0 # Shannon entropy of P_i
        denom = sum([np.exp(-self.get_dist(X, i, k)/(2 * sigma_i**2)) for k in range(X.shape[0])
                    if k != i])
        for j in range(N):
            if i == j:
                pass
            numerator = np.exp(-self.get_dist(X, i, j)/(2 * sigma_i**2))
            p = numerator/denom    
            if p != 0:
                h -= p * np.log2(p)
#             print("j=%d, p=%f, h=%f" % (j, p, h))
        perp = 2**h
#         print("perplexity = %f" % (perp))
        return perp
        
    def get_joint_p(self, X, i, j):
        """ Return p_{ij}"""
        N = X.shape[0]        
        if self.joint_p is None:
            print("Initializing joint probas matrix")
            self.joint_p = np.zeros((N,N))
            for k,l in itertools.combinations_with_replacement(range(N), 2):
                self.joint_p[k,l] = (self.get_cond_p(k, l) + self.get_cond_p(l, k)) / (2*N)
            self.joint_p = sparse.triu(self.joint_p, format="lil")
            print("Finished computing joint probas matrix.")
        
        if i>j:
            i,j = j,i
        return self.joint_p[i,j]
    
    def compute_joint_q(self, y, i, j):
        numerator = sum([(1+ sum((y[k] - y[l])**2)) 
                        for k,l in itertools.product(range(y.shape[0]), repeat=2)
                        if k != l])
        denom = 1 + sum((y[i] - y[j])**2)
        return numerator/denom
    
    def gradient_cost(self, X, y, i=None):
        N = X.shape[0]
        
        numerator = sum([(1+ sum((y[k] - y[l])**2)) 
                        for k,l in itertools.product(range(y.shape[0]), repeat=2)
                        if k != l])
        
        def grad_i(i):
            s = 0
            for j in range(N):
                denom = 1 + sum((y[i] - y[j])**2)
                s += (self.get_joint_p(X, i, j) -  (numerator/denom)) * \
                     (y[i] - y[j]) / (1 + sum((y[i]-y[j])**2))
            return 4*s

        if i is not None:
            return grad_i(i)
        else :
            apply_results = np.zeros_like(y)
            for i in range(N):
                apply_results[i] = grad_i(i)
            return apply_results
    
        
    
    def fit_transform(self, X):
        """ Return the low-dimensional representation of X."""
        # Reset the matrices:
        self.X         = X
        self.joint_p   = None
        self.distances = None
        self.cond_p    = None
        self.sigma     = None
        # Sample initial solution y from a gaussian distribution
        print("Sample initial solution")
        y = np.random.random((X.shape[0], self.n_components))
        print("Iterate to find the best y")
        for t in range(self.n_iter):
            print("[%5d/%d] Compute the gradient..." % (t, self.n_iter))
            grad = self.gradient_cost(X, y)
            print("Gradient norm: %f, incrementing y" % np.linalg.norm(grad))
            y -= self.learning_rate * grad
        return y
    

from sklearn.utils import shuffle

iris = datasets.load_iris()
X, y = iris.data[:], iris.target[:]
X,y = shuffle(X, y)

# X,y = X[:30], y[:30]
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Classes in y:", np.unique(y, return_counts=True))

tsne = TSNE(n_components = 2, perplexity=30, n_iter=500, n_jobs=10)
X_tsne = tsne.fit_transform(X)
print("Shape of X_tsne:", X_tsne.shape)

plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y)



Shape of X: (150, 4)
Shape of y: (150,)
Classes in y: (array([0, 1, 2]), array([50, 50, 50]))
Sample initial solution
Iterate to find the best y
[    0/500] Compute the gradient...
Initializing joint probas matrix
Initializing conditional probas matrix
Trigger get_dist
Initializing distances matrix
Finished computing distances matrix
Finished computing conditional probas matrix.
Finished computing joint probas matrix.
Gradient norm: 40356333.154892, incrementing y
[    1/500] Compute the gradient...
Gradient norm: 235.869132, incrementing y
[    2/500] Compute the gradient...
Gradient norm: 231.542531, incrementing y
[    3/500] Compute the gradient...
Gradient norm: 227.434264, incrementing y
[    4/500] Compute the gradient...
Gradient norm: 223.527287, incrementing y
[    5/500] Compute the gradient...
Gradient norm: 219.806310, incrementing y
[    6/500] Compute the gradient...
Gradient norm: 216.257577, incrementing y
[    7/500] Compute the gradient...
Gradient norm: 212.868676, 

Gradient norm: 106.801855, incrementing y
[  101/500] Compute the gradient...
Gradient norm: 106.367954, incrementing y
[  102/500] Compute the gradient...
Gradient norm: 105.939149, incrementing y
[  103/500] Compute the gradient...
Gradient norm: 105.515340, incrementing y
[  104/500] Compute the gradient...
Gradient norm: 105.096430, incrementing y
[  105/500] Compute the gradient...
Gradient norm: 104.682324, incrementing y
[  106/500] Compute the gradient...
Gradient norm: 104.272931, incrementing y
[  107/500] Compute the gradient...
Gradient norm: 103.868161, incrementing y
[  108/500] Compute the gradient...
Gradient norm: 103.467928, incrementing y
[  109/500] Compute the gradient...
Gradient norm: 103.072146, incrementing y
[  110/500] Compute the gradient...
Gradient norm: 102.680733, incrementing y
[  111/500] Compute the gradient...
Gradient norm: 102.293609, incrementing y
[  112/500] Compute the gradient...
Gradient norm: 101.910696, incrementing y
[  113/500] Compute th

### Question 2
Construisez des visualisation (en 2D et en 3D) des jeux de données proposés (ou autres). 

Comparez avec d’autres méthodes de réduction de dimension, linéaires et non-linéaires (voir http://scikit-learn.org/stable/modules/classes.html#module-sklearn.manifold).



In [ ]:
from sklearn import datasets
from sklearn import manifold

iris = datasets.load_iris()
X, y = iris.data, iris.target

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Classes in y:", np.unique(y, return_counts=True))

skl_tsne = manifold.TSNE(n_components = 2)
X_tsne = skl_tsne.fit_transform(X, y)
print("Shape of X_tsne:", X_tsne.shape)

plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y)



Question 3
Pour Isomap et LLE, manipulez le paramètre contrôlant la taille du voisinage pour la construction du
graphe.
Exercice 2 Recommandation
Jeu de données MovieLens http://files.grouplens.org/datasets/movielens/ :
- 100k http://files.grouplens.org/datasets/movielens/ml-100k.zip
- 1M http://files.grouplens.org/datasets/movielens/ml-1m.zip
Question 1
Construisez une visualisation avec t-SNE (on peut utiliser metric=precomputed pour donner une matrice
de distance au lieu des coordonnées des points en entrée de la fonction sklearn).
1
Question 2
Implémentez un modèle de collaborative filtering avec :
- descente de gradient stochastique
- distance L2
- régularisation L2
- sans puis avec biais
Question 3
Évaluez vos modèles sur MovieLens.